In [42]:
import pdal
import json
import geopandas
import shapely

In [43]:
crs=9754

In [44]:
pdal_pipeline_instructions = [
    {
        "type": "readers.text",
        "filename": r"C:\Local\data\GNS23201\xyzZo3_1.txt",
    },
    {
        "type":"writers.las",
        "filename":r"C:\Local\data\GNS23201\xyzZo3_1.las",
        #"a_srs": f"EPSG:{crs}"
    }
]

In [45]:
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()

2221044

In [46]:
points = pdal_pipeline.arrays[0]

In [47]:
x0 = points['X'].min(); x1 = points['X'].max(); y0 = points['Y'].min(); y1 = points['Y'].max()
boundary = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0, y0], [x0, y1], [x1, y1], [x1, y0]])], crs=crs)
boundary.to_file(rf"C:\Local\data\GNS23201\epsg{crs}.geojson")

In [49]:
pdal_pipeline_instructions = [ r"C:\Local\data\GNS23201\xyzZo3_1.las",
    { "type": "filters.delaunay" },
    { "type": "filters.faceraster", "resolution": 1 },
    { "type": "writers.raster", "filename": r"C:\Local\data\GNS23201\xyzZo3_1.tif" }
]

In [50]:
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()

2221044